# Recommender Systems - Data Mining Lab 2

In [ ]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

### Data loading

In [ ]:
train_file_path = 'data/lab2_train.csv' 
test_file_path = 'data/lab2_test.csv'

train_data = pd.read_csv(train_file_path, delimiter=',')
test_data = pd.read_csv(test_file_path, delimiter=',')


In [ ]:
#print(train_data)
#print(like_train_data)
like_train_data = train_data[train_data.is_like == True]
like_train_data = like_train_data.sort_values("user_to_id")
match_train_data = train_data[train_data.is_match ==True]
match_train_data = match_train_data.sort_values("user_from_id")

liked_user_count = like_train_data["user_to_id"].nunique()
user_count = like_train_data["user_from_id"].nunique()
match_count = match_train_data["user_from_id"].nunique()


number_of_likes = like_train_data["user_to_id"]
counts = number_of_likes.value_counts()
plt.figure(figsize=(15, 8))
plt.bar(counts.index, counts.values, color='blue')

plt.xlabel('User IDs')
plt.ylabel('Number of Likes')
plt.title('Distribution of Number of Likes per User ID')
plt.xticks([])  # Hides x axis labels due to the large number of ids
plt.show()

#print(user_count)
#print(liked_user_count)
#print(match_count)

#print(like_train_data[like_train_data.user_to_id == 36])
#print(like_train_data[like_train_data.user_from_id == 36])


#print(like_train_data)

#print(match_train_data)

## Familiarization
### Important properties of the data
There are only 1707 user who have one like or more of the 2107 users who gave likes. Of all of these users there are only 415 matches. Of these matches there are 345 unique users who receive a match. So there is only a small group of users who receive more than one match. And a small number of matches on the total of 76392 likes and dislikes.

The Data is therefore very sparse. Of these 76392 likes and dislikes there are only 12637 likes. This is 16,5% likes. Of these likes there are only 415 matches. Thus only 3,3% mutual likes. This is sparse data to work with.   

### Types of people

- Someone who receives a lot of likes but likes almost no one. User 2 receives 45 likes but likes none of them. Therefore not leading to a match

- Someone who receives and gives about only dislikes. For example user 28 is very active on the platform but gives about only dislikes and receives only dislikes. Therefore not leading to a match

- Someone who gives a lot of likes and receives only dislikes. For example user 18 gives 26 likes and receives no likes back and has thus not a match.

- Someone who receives a lot of likes and gives a lot of likes. For example user 36 gives a lot of likes and receives a lot of likes and eventualy leading to a match.

## Non-negative matrix factorization

## Distance-based pipeline

## Report on code